In [1]:
# design goals:
#
# - understandability
# - modularity
# - configurability

In [ ]:
# config

WORKSPACE_DIR="workspaces/default"
# replace with pathlib

# mkdir etc

In [ ]:
# conversion

CONVERSION_OUTPUT_DIR = f"{WORKSPACE_DIR}/conversion"
# replace with pathlib

In [ ]:
# chunking

In [ ]:
# authoring

# TODO: plug into docs

filters = [
    lambda chunk: len(str(chunk.text)) > 500
]

dataset = {}
for doc in docs:
    print(f"Chunking and filtering document {doc.document.name}")

    chunks = list(chunker.chunk(dl_doc=doc.document))
    qa_chunks = list(get_qa_chunks(doc.document.name, chunks, filters))
    dataset[doc.document.name] = qa_chunks
    
    print(f"Created dataset {doc.document.name} with {len(qa_chunks)} QA chunks")

#Initialize QA generator, supplying details for which model to use

GenerateOptions controls which model is used for QA generation by setting generate_options.provider below. Three options are available:

* LlmProviders.WATSONX for watsonx
* LlmProviders.OPENAI for OpenAI
* LlmProviders.OPENAI_LIKE for any model provider with OpenAI compatible APIs

In [ ]:
!pip install docling-sdg

In [ ]:
from docling_sdg.qa.generate import Generator
from docling_sdg.qa.base import GenerateOptions, LlmProviders

generate_options = GenerateOptions(api_key="fake", project_id="project_id")
generate_options.provider = LlmProviders.OPENAI_LIKE
generate_options.api_key = "fake"
generate_options.model_id = "mixtral" # for local ollama
generate_options.generated_file = f"data/chunks-{filename_base}.jsonl"

gen = Generator(generate_options=generate_options)

In [ ]:
for doc, chunks in dataset.items():
    print(f"processing chunks that looks like:\n{chunks[0].text}")
    results = gen.generate_from_chunks(chunks)
    print(f"{doc}: {results.status}")
    break

# Read generated QAs and restructure

In [ ]:
import json
import yaml
from textwrap import wrap

qnas = {}
chunk_id_to_text = {}
with open(generate_options.generated_file, "rt") as f:
    for line in f.readlines():
        entry = json.loads(line)
        chunk_id = entry['chunk_id']
        if chunk_id not in chunk_id_to_text:
            chunk_id_to_text[chunk_id] = entry['context']
        if chunk_id not in qnas:
            qnas[chunk_id] = []
        qnas[chunk_id].append({'question': entry['question'], 'answer': entry['answer']})

print(qnas[:1]

# Output qna.yaml

In [ ]:
def str_presenter(dumper, data):
  if len(data.splitlines()) > 1:  # check for multiline string
    return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
  elif len(data) > 80:
    data = "\n".join(wrap(data, 80))
    return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
  return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

# to use with safe_dump:
yaml.representer.SafeRepresenter.add_representer(str, str_presenter)

class IndentedDumper(yaml.Dumper):
    def increase_indent(self, flow=False, indentless=False):
        return super(IndentedDumper, self).increase_indent(flow, False)

data = {'seed_examples': []}
for chunk_id, context in chunk_id_to_text.items():
    data['seed_examples'].append({
        'context': context,
        'questions_and_answers': [
            {
                'question': example['question'],
                'answer': example['answer'],
            } for example in qnas[chunk_id]
        ]
    })

with open('qna.yml', 'w') as yaml_file:
    yaml.dump(data, yaml_file, Dumper=IndentedDumper, default_flow_style=False, sort_keys=False, width=80)

print("Done")

In [ ]:
# sdg